## From Tables to Graphs: Data Ingestion into Neo4j with Cypher

### A hands-on walkthrough for converting tabular data into a connected graph model using Neo4j and Cypher.


## Introduction

Graphs are everywhere — from social networks and supply chains to biological pathways and knowledge graphs. 
But real-world datasets often start in tabular form (CSV, Excel), and to unleash the power of graph analytics, 
we need to model and ingest this data into a graph database.

In this notebook, we'll:
1. Load tabular data into Dataframe and cleaning
2. Neo4J Setup & Environment
3. Design a graph schema.
4. Ingest data into Neo4j using Cypher queries.
5. Run queries to validate our ingestion.
6. Create Vector Index on Graph DB

**Tools:**
- **Neo4j**: A leading graph database
- **pandas**: For reading and transforming tabular data
- **OpenAI Embeddings**: For creating entity embeddings

### 1. Load tabular data into Dataframe and cleaning

In [3]:
import pandas as pd

In [4]:
adonis_df = pd.read_excel('data/FINANCE_REPORTING_PROCESS_MAPS.xlsx')
adonis_df.shape

(3377, 16)

In [6]:
adonis_df.head()

,Step Type,Step Name,Model (Process Diagram),Step (Order),Description,Task Type,Responsible for execution (Role),Referenced Event,Cross-reference,Referenced Subprocess,Process Area,Sub Process Area,Implemented Controls (NFCM Controls),Frequency of control execution,Implemented Controls (NFCM Controls) - Description,Implemented Controls (NFCM Controls) - Default Relevance
0,Cross-reference,Not Allocated Service Charges(NASC),1.10.1.1 Intercompany Accounting,NaN,NaN,NaN,NaN,NaN,"1.46.9, Not Allocated Service Charges",NaN,R2R Landing Pages,NaN,NaN,NaN,NaN,NaN
1,Task,Select internal orders/WBS elements variant to...,"1.48.16, Settlement of Internal orders/ WBS el...",3.0,All of the settlement requests must be reviewe...,Manual,R2R Specialist,NaN,NaN,NaN,GL Management & Reporting,Manage Period End Close,NaN,NaN,NaN,NaN
2,Subprocess,Balance Sheet Reconciliations (non-Blackline e...,"1.46.11, Intercompany Transfer of Goods",32.0,NaN,NaN,NaN,NaN,NaN,"1.48.13 , Balance Sheet Reconciliations (non-...",Intercompany Accounting,Process Intercompany Invoice,NaN,NaN,NaN,NaN
3,Exclusive Gateway,Is new service type requested?,"1.46.12, Manage Service Type",6.0,NaN,NaN,NaN,NaN,NaN,NaN,Intercompany Accounting,Manage Intercompany Setup,NaN,NaN,NaN,NaN
4,Task,Record on FCRS BS31 schedule in accordance wit...,"1.48.72, Employees Shares Purchase Program (ap...",19.0,Record on FCRS BS31 schedule in accordance wit...,Manual,R2R Specialist,NaN,NaN,NaN,HR Accounting,Share Based Compensation,NaN,NaN,NaN,NaN


In [7]:
adonis_df.isnull().sum()

Step Type                                                      0
Step Name                                                      0
Model (Process Diagram)                                        0
Step (Order)                                                1195
Description                                                 2687
Task Type                                                   2269
Responsible for execution (Role)                            2323
Referenced Event                                            3252
Cross-reference                                             2681
Referenced Subprocess                                       3268
Process Area                                                   4
Sub Process Area                                            1169
Implemented Controls (NFCM Controls)                        3263
Frequency of control execution                              3272
Implemented Controls (NFCM Controls) - Description          3272
Implemented Controls (NFC

In [8]:
adonis_df = adonis_df.replace({pd.NA: ""})
adonis_df["Step (Order)"].fillna(0, inplace=True)

C:\Users\NALABSR1\AppData\Local\Temp\ipykernel_34364\632813227.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  adonis_df["Step (Order)"].fillna(0, inplace=True)


In [9]:
adonis_df.isnull().sum()

Step Type                                                   0
Step Name                                                   0
Model (Process Diagram)                                     0
Step (Order)                                                0
Description                                                 0
Task Type                                                   0
Responsible for execution (Role)                            0
Referenced Event                                            0
Cross-reference                                             0
Referenced Subprocess                                       0
Process Area                                                0
Sub Process Area                                            0
Implemented Controls (NFCM Controls)                        0
Frequency of control execution                              0
Implemented Controls (NFCM Controls) - Description          0
Implemented Controls (NFCM Controls) - Default Relevance    0
dtype: i

In [10]:
adonis_df.columns.to_list()

['Step Type',
 'Step Name',
 'Model (Process Diagram)',
 'Step (Order)',
 'Description',
 'Task Type',
 'Responsible for execution (Role)',
 'Referenced Event',
 'Cross-reference',
 'Referenced Subprocess',
 'Process Area',
 'Sub Process Area',
 'Implemented Controls (NFCM Controls)',
 'Frequency of control execution',
 'Implemented Controls (NFCM Controls) - Description',
 'Implemented Controls (NFCM Controls) - Default Relevance']

### 2. Neo4J Setup & Environment

In [11]:
from neo4j import GraphDatabase

NEO4J_URI = "bolt://localhost:7687" # os.getenv("NEO4J_URI")
NEO4J_USER = "neo4j" # os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = "Neo4j@123" # os.getenv("NEO4J_PASSWORD")

# NEO4J_URI = "bolt://13.235.135.217:7687"
# NEO4J_USER = "neo4j"
# NEO4J_PASSWORD = ""

AUTH = (NEO4J_USER, NEO4J_PASSWORD)
with GraphDatabase.driver(NEO4J_URI, auth=AUTH) as driver:
    driver.verify_connectivity()


KeyboardInterrupt



In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
    
# Load environment variables from .env file
load_dotenv()

DEPLOYMENT_NAME = "gpt-4.1"
EMBEDDING_DEPLOYMENT_NAME = "text-embedding-3-small"
  
embedding_model = OpenAIEmbeddings(model=EMBEDDING_DEPLOYMENT_NAME)

### 3. Design a graph schema.

In [16]:
import re

# Keep regular embeddings for non-summary text
def get_embedding(text: str) -> list:
    text = (text or "").strip().replace("\n", " ")
    return embedding_model.embed_query(text)

def parse_control_descriptions(desc_block: str):
    """Parses control descriptions into a dict: {control_name: description_text}"""
    descriptions = {}
    pattern = re.compile(r'(?P<name>NFCM\.[\w\.]+) \(Control\):\s*(?P<desc>.*?)(?=\nNFCM\.|\Z)', re.DOTALL)
    for match in pattern.finditer((desc_block or "").strip()):
        name = match.group('name').strip()
        desc = match.group('desc').strip().replace('\n', ' ')
        descriptions[name] = desc
    return descriptions

def insert_step_and_relationships(tx, row):
    # Parse responsible roles (Accountable column removed per request)
    responsible_roles = [
        r.strip()
        for r in re.split(r'\s*(?:&|\bor\b)\s*|,|/', str(row.get("Responsible for execution (Role)", "")))
        if r.strip()
    ]

    # Text for embeddings (NO summary embeddings; Function column removed)
    process_area_text = row.get("Process Area", "")
    sub_process_area_text = row.get("Sub Process Area", "")
    diagram_text = row.get("Model (Process Diagram)", "")
    step_desc_text = row.get("Description", "")
    control_desc_text = row.get("Control - Description", "")

    # Generate embeddings for NON-summary fields only
    embeddings = {
        "process_area": get_embedding(process_area_text),
        "sub_process_area": get_embedding(sub_process_area_text),
        "diagram": get_embedding(diagram_text),
        "step": get_embedding(step_desc_text),
        "control": get_embedding(control_desc_text),
    }

    # Insert core nodes & hierarchy (NO summaries anywhere)
    tx.run("""
        MERGE (pa:ProcessArea {name: $processArea})
        SET pa.embedding = $process_area_emb

        MERGE (spa:SubProcessArea {name: $subProcessArea})
        SET spa.embedding = $sub_process_area_emb

        MERGE (pd:ProcessDiagram {name: $processDiagram})
        SET pd.embedding = $diagram_emb

        MERGE (s:Step {
            name: $stepName,
            diagram: $processDiagram
        })
        SET s.stepType = $stepType,
            s.stepOrder = $stepOrder,
            s.description = $description,
            s.taskType = $taskType,
            s.referencedEvent = $referencedEvent,
            s.crossReference = $crossReference,
            s.referencedSubprocess = $referencedSubprocess,
            s.embedding = $step_emb

        MERGE (c:NFCMControl {name: $control})
        SET c.description = $controlDescription,
            c.frequencyExecution = $frequencyExecution,
            c.relevance = $controlRelevance,
            c.embedding = $control_description_emb

        MERGE (c)-[:INVOLVED_IN]->(s)
        MERGE (s)-[:STEP_IN]->(pd)
        MERGE (pd)-[:PART_OF]->(spa)
        MERGE (spa)-[:PART_OF]->(pa)
    """, {
        # Required hierarchy fields (Function removed)
        "processArea": row.get("Process Area", ""),
        "subProcessArea": row.get("Sub Process Area", ""),
        "processDiagram": row.get("Model (Process Diagram)", ""),

        # Step core fields (columns removed: page_url, Classification, Referenced Documents, Text)
        "stepName": row.get("Step Name", ""),
        "stepType": row.get("Step Type", ""),
        "stepOrder": row.get("Step (Order)"),
        "description": row.get("Description", ""),
        "taskType": row.get("Task Type", ""),
        "referencedEvent": row.get("Referenced Event", ""),
        "crossReference": row.get("Cross-reference", ""),
        "referencedSubprocess": row.get("Referenced Subprocess", ""),

        # Control (columns removed: Control method, Implemented Controls - Title)
        "control": row.get("Control", ""),
        "controlDescription": row.get("Control - Description", ""),
        "frequencyExecution": row.get("Frequency of control execution"),
        "controlRelevance": row.get("Control - Default Relevance", ""),

        # Embeddings for NON-summary text
        "process_area_emb": embeddings["process_area"],
        "sub_process_area_emb": embeddings["sub_process_area"],
        "diagram_emb": embeddings["diagram"],
        "step_emb": embeddings["step"],
        "control_description_emb": embeddings["control"],
    })

    # Responsible Roles (keep embeddings)
    for role in responsible_roles:
        tx.run("""
            MERGE (r:Role {name: $role})
            SET r.embedding = $embedding
            MERGE (s:Step {name: $stepName, diagram: $processDiagram})
            MERGE (r)-[:RESPONSIBLE_FOR]->(s)
        """, {
            "role": role,
            "embedding": get_embedding(role),
            "stepName": row.get("Step Name", ""),
            "processDiagram": row.get("Model (Process Diagram)", "")
        })

    # 🔗 Inter-ProcessDiagram references FROM Step TO ProcessDiagram
    step_type = str(row.get("Step Type", "")).strip()
    source_step = row.get("Step Name", "")
    source_diagram = row.get("Model (Process Diagram)", "")

    ref_event = str(row.get("Referenced Event", "") or "").strip()
    cross_ref = str(row.get("Cross-reference", "") or "").strip()
    ref_subproc = str(row.get("Referenced Subprocess", "") or "").strip()

    if step_type == "Intermediate Event (sequence flow)" and ref_event:
        tx.run("""
            MERGE (s:Step {name: $stepName, diagram: $sourceDiagram})
            MERGE (tgt:ProcessDiagram {name: $targetDiagram})
            MERGE (s)-[:REFERENCED_EVENT]->(tgt)
        """, {
            "stepName": source_step,
            "sourceDiagram": source_diagram,
            "targetDiagram": ref_event
        })

    elif step_type == "Cross-reference" and cross_ref:
        tx.run("""
            MERGE (s:Step {name: $stepName, diagram: $sourceDiagram})
            MERGE (tgt:ProcessDiagram {name: $targetDiagram})
            MERGE (s)-[:CROSS_REFERENCE]->(tgt)
        """, {
            "stepName": source_step,
            "sourceDiagram": source_diagram,
            "targetDiagram": cross_ref
        })

    elif step_type == "Subprocess" and ref_subproc:
        tx.run("""
            MERGE (s:Step {name: $stepName, diagram: $sourceDiagram})
            MERGE (tgt:ProcessDiagram {name: $targetDiagram})
            MERGE (s)-[:REFERENCED_SUBPROCESS]->(tgt)
        """, {
            "stepName": source_step,
            "sourceDiagram": source_diagram,
            "targetDiagram": ref_subproc
        })

def insert_adonis_data(df):
    with driver.session() as session:
        for _, row in df.iterrows():
            session.write_transaction(insert_step_and_relationships, row)


### 4. Ingest data into Neo4j using Cypher queries.

In [17]:
insert_adonis_data(adonis_df)

C:\Users\NALABSR1\AppData\Local\Temp\ipykernel_25064\1701601612.py:205: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\NALABSR1\AppData\Local\Temp\ipykernel_25064\1701601612.py:207: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_step_and_relationships, row)


### 5. Run queries to validate our ingestion.

In [13]:
# Example query to fetch all companies and their departments
query = """
MATCH (n:NFCMControl) WHERE n.frequency = 'Quarterly' RETURN n.name, n.method
"""
with driver.session() as session:
    result = session.run(query)
    for record in result:
        print(record["Company"], ":", record["Departments"])

C:\Users\NALABSR1\AppData\Local\Temp\ipykernel_27068\3709761944.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] No connection could be made because the target machine actively refused it)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] No connection could be made because the target machine actively refused it)

### 6. Create Vector Index

In [3]:
with driver.session() as session:
    session.run("""
    MATCH (n:Step) WHERE n.embedding IS NOT NULL SET n:Searchable;
    MATCH (n:ProcessDiagram) WHERE n.embedding IS NOT NULL SET n:Searchable;
    MATCH (n:Role) WHERE n.embedding IS NOT NULL SET n:Searchable;
    MATCH (n:NFCMControl) WHERE n.embedding IS NOT NULL SET n:Searchable;
    MATCH (n:Function) WHERE n.embedding IS NOT NULL SET n:Searchable;
    MATCH (n:ProcessArea) WHERE n.embedding IS NOT NULL SET n:Searchable;
    MATCH (n:SubProcessArea) WHERE n.embedding IS NOT NULL SET n:Searchable;
""")

In [ ]:
with driver.session() as session:
    session.run("""
    CREATE VECTOR INDEX adonis_graph_embedding_index IF NOT EXISTS
    FOR (n:Searchable) ON (n.embedding)
    OPTIONS {
      indexConfig: {
        `vector.dimensions`: 1536,
        `vector.similarity_function`: 'cosine'
      }
    };"""
    )